# BERT Model: Fine-tuned Transformer for Fake News Classification

This notebook implements a fine-tuned BERT model for binary classification of fake vs real news articles.

## 🔧 Steps:
1. Import libraries and load data
2. Minimal preprocessing (preserve structure for BERT)
3. BERT tokenization and encoding
4. Model setup and fine-tuning
5. Training with validation
6. Evaluation and comparison

## ✅ Purpose:
Achieve state-of-the-art performance using transformer architecture (~85-90% accuracy expected).

## 1. Import Libraries and Setup

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import time
import warnings
warnings.filterwarnings('ignore')

# Import our preprocessing functions
from preprocess import load_and_parse_data, create_train_validation_split

# Check if MPS (Metal Performance Shaders) is available for M4
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) for M4 acceleration")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA GPU")
else:
    device = torch.device("cpu")
    print("Using CPU (this will be slower)")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

## 2. Load and Parse Data

Using our preprocessing module to load the tab-separated data format.

In [ ]:
# Load data using our preprocessing function
print("Loading training data...")
train_data = load_and_parse_data('data/training_data_lowercase.csv')

print("Loading test data...")
test_data = load_and_parse_data('data/testing_data_lowercase_nolabels.csv')

# Convert to DataFrames for easier handling
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Label distribution: {Counter(train_df['label'])}")

# Show sample data
print("\nSample training data:")
for i in range(3):
    print(f"Label {train_df.iloc[i]['label']}: {train_df.iloc[i]['text'][:80]}...")

## 3. Minimal Text Preprocessing

BERT works best with minimal preprocessing - we'll only clean essential formatting issues while preserving punctuation and structure.

In [ ]:
def minimal_bert_cleaning(text):
    """Minimal cleaning for BERT - preserve original structure"""
    if pd.isna(text) or text == '':
        return ""
    
    # Convert to string and remove excessive whitespace only
    text = str(text).strip()
    text = ' '.join(text.split())  # Remove extra spaces
    
    return text

# Apply minimal cleaning
train_df['clean_text'] = train_df['text'].apply(minimal_bert_cleaning)
test_df['clean_text'] = test_df['text'].apply(minimal_bert_cleaning)

# Remove any empty texts
train_df = train_df[train_df['clean_text'].str.len() > 0].reset_index(drop=True)

# Analyze text lengths for optimal max_length
text_lengths = train_df['clean_text'].str.split().str.len()
print(f"Text length statistics (words):")
print(f"Mean: {text_lengths.mean():.1f}")
print(f"Median: {text_lengths.median():.1f}")
print(f"95th percentile: {text_lengths.quantile(0.95):.1f}")
print(f"99th percentile: {text_lengths.quantile(0.99):.1f}")

# Choose max_length based on 95th percentile + buffer
max_length = min(256, int(text_lengths.quantile(0.95)) + 20)
print(f"\nUsing max_length: {max_length}")

## 4. BERT Tokenization and Dataset Creation

Setting up BERT tokenizer and creating PyTorch datasets for training.

In [ ]:
# Initialize BERT tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"Using model: {model_name}")
print(f"Tokenizer vocabulary size: {len(tokenizer.vocab)}")

class NewsDataset(Dataset):
    """Custom dataset for news classification"""
    
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        
        # Tokenize text
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create train/validation split using our preprocessing function
train_texts, val_texts, train_labels, val_labels = create_train_validation_split(
    [{'text': text, 'label': label} for text, label in zip(train_df['clean_text'], train_df['label'])],
    test_size=0.2,
    random_state=42
)

# Create datasets
train_dataset = NewsDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = NewsDataset(val_texts, val_labels, tokenizer, max_length)

print(f"\nDataset sizes:")
print(f"Training: {len(train_dataset)}")
print(f"Validation: {len(val_dataset)}")

# Test tokenization on a sample
sample = train_dataset[0]
print(f"\nSample tokenization shape:")
print(f"Input IDs: {sample['input_ids'].shape}")
print(f"Attention mask: {sample['attention_mask'].shape}")
print(f"Label: {sample['labels']}")

## 5. Model Setup and Training Configuration

Loading pre-trained BERT and setting up training parameters for fine-tuning.

In [ ]:
# Load pre-trained BERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)

# Move model to device
model.to(device)

print(f"Model loaded with {model.num_parameters():,} parameters")
print(f"Model device: {next(model.parameters()).device}")

# Define training arguments
training_args = TrainingArguments(
    output_dir='./bert_results',
    num_train_epochs=3,              # Start with 3 epochs
    per_device_train_batch_size=16,  # Adjust based on memory
    per_device_eval_batch_size=32,   # Larger batch for evaluation
    warmup_steps=500,                # Warmup for learning rate
    weight_decay=0.01,               # Regularization
    logging_dir='./bert_logs',
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=500,                  # Evaluate every 500 steps
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
    report_to=None,                  # Disable wandb/tensorboard
    seed=42
)

# Define metrics computation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

print("\nTraining setup complete!")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Number of epochs: {training_args.num_train_epochs}")
print(f"Learning rate: {training_args.learning_rate}")

## 6. Model Training

Fine-tuning BERT on our fake news dataset with validation monitoring.

In [ ]:
# Start training
print("Starting BERT fine-tuning...")
print(f"Training on {len(train_dataset)} samples")
print(f"Validating on {len(val_dataset)} samples")
print(f"Using device: {device}")

start_time = time.time()

# Train the model
train_result = trainer.train()

training_time = time.time() - start_time

print(f"\nTraining completed!")
print(f"Training time: {training_time/60:.2f} minutes")
print(f"Final training loss: {train_result.training_loss:.4f}")

# Save the trained model
trainer.save_model('./bert_fine_tuned')
print("\nModel saved to './bert_fine_tuned'")

## 7. Model Evaluation

Comprehensive evaluation with accuracy, classification report, and confusion matrix.

In [ ]:
# Evaluate on validation set
print("Evaluating model performance...")

# Get predictions
predictions = trainer.predict(val_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = val_labels

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
print(f"\nBERT Model Results:")
print(f"Validation Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Detailed classification report
print("\nClassification Report:")
class_names = ['Fake (0)', 'Real (1)']
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('BERT Model - Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Performance by class
print("\n📈 Performance by Class:")
for i, class_name in enumerate(class_names):
    class_mask = np.array(y_true) == i
    class_accuracy = accuracy_score(np.array(y_true)[class_mask], y_pred[class_mask])
    print(f"{class_name}: {class_accuracy:.4f} ({class_accuracy*100:.2f}%)")

## 8. Model Comparison and Analysis

Comparing BERT performance with baseline and analyzing key insights.

In [ ]:
# Training history analysis
if hasattr(trainer.state, 'log_history'):
    logs = trainer.state.log_history
    
    # Extract training and validation metrics
    train_losses = [log['train_loss'] for log in logs if 'train_loss' in log]
    eval_accuracies = [log['eval_accuracy'] for log in logs if 'eval_accuracy' in log]
    
    if train_losses and eval_accuracies:
        plt.figure(figsize=(12, 4))
        
        # Training loss
        plt.subplot(1, 2, 1)
        plt.plot(train_losses)
        plt.title('Training Loss')
        plt.xlabel('Steps')
        plt.ylabel('Loss')
        plt.grid(True)
        
        # Validation accuracy
        plt.subplot(1, 2, 2)
        plt.plot(eval_accuracies)
        plt.title('Validation Accuracy')
        plt.xlabel('Evaluation Steps')
        plt.ylabel('Accuracy')
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()
        
        print(f"Best validation accuracy: {max(eval_accuracies):.4f}")
        print(f"Final validation accuracy: {eval_accuracies[-1]:.4f}")

# Model comparison summary
print("\n" + "="*50)
print("MODEL COMPARISON SUMMARY")
print("="*50)
print(f"Model: BERT-base-uncased")
print(f"Parameters: {model.num_parameters():,}")
print(f"Training time: {training_time/60:.2f} minutes")
print(f"Validation accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Max sequence length: {max_length}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Device used: {device}")

# Expected performance note
baseline_accuracy = 0.70  # Approximate baseline from logistic regression
improvement = accuracy - baseline_accuracy
print(f"\n🎯 Performance vs Baseline:")
print(f"Baseline (Logistic Regression): ~{baseline_accuracy:.2f}")
print(f"BERT Model: {accuracy:.4f}")
print(f"Improvement: +{improvement:.4f} ({improvement*100:.2f} percentage points)")

## 9. Test Set Predictions (Optional)

Generate predictions for the test set if needed for submission.

In [ ]:
# Uncomment this section if you need test set predictions

# # Create test dataset (without labels)
# test_texts = test_df['clean_text'].tolist()
# test_labels = [0] * len(test_texts)  # Dummy labels for dataset creation
# 
# test_dataset = NewsDataset(test_texts, test_labels, tokenizer, max_length)
# 
# print(f"Generating predictions for {len(test_dataset)} test samples...")
# 
# # Get predictions
# test_predictions = trainer.predict(test_dataset)
# test_pred_labels = np.argmax(test_predictions.predictions, axis=1)
# test_pred_probs = torch.softmax(torch.tensor(test_predictions.predictions), dim=1)
# 
# # Create submission DataFrame
# submission_df = pd.DataFrame({
#     'text': test_texts,
#     'predicted_label': test_pred_labels,
#     'fake_probability': test_pred_probs[:, 0].numpy(),
#     'real_probability': test_pred_probs[:, 1].numpy()
# })
# 
# # Save predictions
# submission_df.to_csv('bert_predictions.csv', index=False)
# print("Test predictions saved to 'bert_predictions.csv'")
# 
# # Show sample predictions
# print("\nSample predictions:")
# print(submission_df[['predicted_label', 'fake_probability', 'real_probability']].head())

print("Test predictions section ready (currently commented out)")
print("Uncomment the code above to generate test set predictions")

## 10. Key Insights and Next Steps

Summary of findings and recommendations for further improvement.

In [ ]:
print("KEY INSIGHTS FROM BERT MODEL:")
print("="*50)

# Model performance insights
if accuracy > 0.85:
    print("Excellent performance: Model achieves high accuracy on fake news detection")
elif accuracy > 0.80:
    print("Good performance: Model shows strong fake news detection capabilities")
elif accuracy > 0.75:
    print("Moderate performance: Room for improvement in fake news detection")
else:
    print("Lower performance: Consider hyperparameter tuning or data quality")

print(f"\nModel Statistics:")
print(f"• Final accuracy: {accuracy:.4f}")
print(f"• Training efficiency: {len(train_dataset)/(training_time/60):.0f} samples/minute")
print(f"• Model size: {model.num_parameters()/1e6:.1f}M parameters")

print(f"\nPotential Improvements:")
print(f"• Increase training epochs (currently {training_args.num_train_epochs})")
print(f"• Experiment with learning rate scheduling")
print(f"• Try ensemble with multiple models")
print(f"• Consider domain-specific fine-tuning")
print(f"• Implement advanced data augmentation")

print(f"\nBusiness Applications:")
print(f"• Real-time fake news detection systems")
print(f"• Social media content moderation")
print(f"• News credibility scoring")
print(f"• Educational fact-checking tools")

print(f"\nBERT Model Implementation Complete!")
print(f"Model ready for deployment and further optimization.")